## Imports

In [52]:
import pyxdf 
import numpy as np
import pandas as pd
from lmfit.models import Model
from scipy.spatial.transform import Rotation as R
from scipy import stats
import pywt
import math

## Function Definitions

Predicted pupil dilation, $d(Y)$, caused by luminance $Y$, is computed with the following equation: $𝑑(𝑌) = 𝑎 · 𝑒^{−𝑏·𝑌} + c$

In [53]:
def pupil_func(x, a, b, c):
    return a * np.exp(-b * x) + c

In [54]:
def modmax(d):
    # compute signal modulus
    m = [0.0]*len(d)
    for i in range(len(d)):
        m[i] = math.fabs(d[i])
    # if value is larger than both neighbours , and strictly larger than either , then it is a local maximum
    t = [0.0]*len(d)
    for i in range(len(d)):
        ll = m[i -1] if i >= 1 else m[i]
        oo = m[i]
        rr = m[i+1] if i < len(d)-2 else m[i]
        if (ll <= oo and oo >= rr) and (ll < oo or oo > rr):
        # compute magnitude
            t[i] = math.sqrt(d[i]**2)
        else:
            t[i] = 0.0
    return t

In [55]:
def ipa_func(d):
    # obtain 2-level DWT of pupil diameter signal d
    try:
        (cA2 ,cD2 ,cD1) = pywt.wavedec(d,'sym16', 'per', level=2)
    except ValueError :
        return
    # get signal duration (in seconds)
    tt = d.index[-1] - d.index[0]
    # normalize by 1/2 j , j = 2 for 2-level DWT
    cA2 [:] = [x / math.sqrt (4.0) for x in cA2]
    cD1 [:] = [x / math.sqrt (2.0) for x in cD1]
    cD2 [:] = [x / math.sqrt (4.0) for x in cD2]

    # detect modulus maxima , see Listing 2
    cD2m = modmax(cD2)
    # threshold using universal threshold λuniv = σˆp(2logn)
    # where σˆ is the standard deviation of the noise
    λuniv = np.std(cD2m) * math.sqrt (2.0* np.log2(len(cD2m )))
    cD2t = pywt. threshold (cD2m ,λuniv,mode="hard")
    # compute IPA
    ctr = 0
    for i in range(len(cD2t )):
        if math.fabs(cD2t[i]) > 0: ctr += 1
    IPA = float(ctr)/tt.total_seconds()

    return IPA

In [56]:
def import_data(file):
    streams, header = pyxdf.load_xdf(file)
    dfs = {}
    for stream in streams:
        stream_name = stream['info']['name'][0]
        stream_channels = {channel['label'][0]: i for i, channel in enumerate(stream['info']['desc'][0]['channels'][0]['channel'])}
        stream_data = stream['time_series']
        data_dict = {key: np.array(stream_data)[:, index] for key, index in stream_channels.items()}
        data_dict['time'] = np.round(np.array(stream['time_stamps']), decimals=4)
        dfs[stream_name] = pd.DataFrame(data_dict).drop_duplicates(subset=['time']).reset_index(drop=True)
    return dfs

In [57]:
accom_time = pd.to_timedelta(0.5, unit='s')

In [58]:
def process_gaze_luminance_data(stream_df):
    pupil = stream_df['GazeStream'].loc[(stream_df['GazeStream']['LeftEyeIsBlinking'] == 0) 
                                        & (stream_df['GazeStream']['RightEyeIsBlinking'] == 0) 
                                        & (stream_df['GazeStream']['LeftPupilDiameter'] > 0) 
                                        & (stream_df['GazeStream']['RightPupilDiameter'] > 0), 
                                        ['time', 'MethodID', 'ModelID', 'LeftPupilDiameter', 'RightPupilDiameter']]
    pupil['time'] = pd.to_timedelta(pupil['time'], unit='s')

    lum = stream_df['LuminanceStream'].loc[:, ['time', 'MethodID', 'ModelID', 'Luminance']]
    lum['time'] = pd.to_timedelta(lum['time'], unit='s')

    # Intersection of time stamps
    pupil_lum_time_intersection = np.intersect1d(pupil['time'], lum['time'])

    # Filter pupil and luminance data by intersection
    pupil = pupil[pupil['time'].isin(pupil_lum_time_intersection)].reset_index(drop=True)
    lum = lum[lum['time'].isin(pupil_lum_time_intersection)].reset_index(drop=True)

    # Combined DataFrame for pupil and luminance
    pupil_lum = pd.DataFrame({
        'time': pd.to_timedelta(pupil_lum_time_intersection, unit='s'),
        'luminance': lum['Luminance'],
        'pupilDiameter': 0.5 * (pupil['LeftPupilDiameter'] + pupil['RightPupilDiameter']),
        'methodID': pupil['MethodID'],
        'modelID': pupil['ModelID']
    })

    return pupil_lum

In [59]:
def process_calibration_data(pupil_lum_df, stream_df):
    calibration_events = stream_df['ExperimentStream'].loc[(stream_df['ExperimentStream']['EventType'] == 'CalibrationColorChange') | 
                                                           (stream_df['ExperimentStream']['SceneEvent'] == 'Calibration') | 
                                                           (stream_df['ExperimentStream']['SceneEvent'] == 'CalibrationComplete'), 
                                                           ['time','SceneEvent', 'EventType']]
    calibration_events['time'] = pd.to_timedelta(calibration_events['time'], unit='s')
    c_start_times = calibration_events[:8]['time']
    c_end_times = calibration_events[1:]['time']
    c_start_times.reset_index(drop=True, inplace=True)
    c_end_times.reset_index(drop=True, inplace=True)

    calib_data = {}
    for i in range(8):
        calib_data[i] = pupil_lum_df.loc[(pupil_lum_df['time'] >= c_start_times[i]) & (pupil_lum_df['time'] <= c_end_times[i]), ['time','luminance', 'pupilDiameter']]
        calib_data[i]['time'] -= calib_data[i]['time'].iloc[0]
        calib_data[i] = calib_data[i].loc[(calib_data[i]['time'] >= accom_time), ['luminance', 'pupilDiameter']]

    calibration_data = pd.concat(calib_data).groupby(level=0).mean().sort_values(by=['luminance']).reset_index(drop=True)
    return calibration_data

In [60]:
def process_navigation_data(pupil_lum_df, stream_df, a, b, c):
    grouped_data = stream_df['NavigationStream'].groupby(['ModelID', 'MethodID'])

    stream_df['SurveyStream']['ModelID'] = stream_df['SurveyStream']['ModelID'].astype(float)
    stream_df['SurveyStream']['MethodID'] = stream_df['SurveyStream']['MethodID'].astype(float)
    
    discomfort_survey = stream_df['SurveyStream'].loc[
        (stream_df['SurveyStream']['SurveyType'] == 'Discomfort') & 
        (stream_df['SurveyStream']['ModelID'] < 4), 
        ['time', 'ModelID', 'MethodID']]
    survey_group = discomfort_survey.groupby(['ModelID', 'MethodID'])

    start_times = []
    end_times = []

    for i in range(4):
        for j in range(2,4):
            trial = grouped_data.get_group((i, j))

            start = trial.loc[(trial['spline_percent'] > 0.001)].index[0]
            start_time = pd.to_timedelta(stream_df['NavigationStream'].loc[start, 'time'], unit='s')

            end = trial.loc[(trial['spline_percent'] > 0.995)]
            end_time = 0
            # For 6DoF navigation, completion was determined by collision with bounding box
            # Spline percentage was based on projection, so it may not reach > 0.995
            # In this case, the survey time serves as the end time
            if len(end) > 0:
                end = end.index[0]
                end_time = pd.to_timedelta(stream_df['NavigationStream'].loc[end, 'time'], unit='s')
            else:
                end = survey_group.get_group((i, j)).index[0]
                end_time = pd.to_timedelta(stream_df['SurveyStream'].loc[end, 'time'], unit='s') - pd.offsets.Second(3)
            
            start_times.append(start_time)
            end_times.append(end_time)


    nav_start_times = start_times
    #nav_start_times.reset_index(drop=True, inplace=True)

    nav_end_times = end_times
    #nav_end_times.reset_index(drop=True, inplace=True)

    nav_data = {}
    for i in range(8):
        nav_data[i] = pupil_lum_df.loc[
            (pupil_lum_df['luminance'] >0) & 
            (pupil_lum_df['time']>nav_start_times[i]) & 
            (pupil_lum_df['time']<nav_end_times[i]), 
            ['time', 'methodID', 'modelID', 'luminance', 'pupilDiameter']]
        nav_data[i].reset_index(drop=True, inplace=True)

    navigation_data = pd.concat(nav_data, names=['trial'])
    navigation_data['pupil_lum_base'] = pupil_func(navigation_data['luminance'], a, b, c)
    navigation_data['adj_pupil'] = navigation_data['pupilDiameter'] - navigation_data['pupil_lum_base']

    return navigation_data

In [61]:
def process_creation_data(pupil_lum_df, stream_df, a, b, c):
    
    crt_start_times = stream_df['CreationStream'].loc[
    (stream_df['CreationStream']['EventName'] == 'StartPointRegistered'), 
    ['time', 'ModelID', 'MethodID']]
    crt_start_times = pd.to_timedelta(crt_start_times.groupby(['ModelID', 'MethodID']).first()['time'], unit='s') + pd.offsets.Second(2)
    crt_start_times.reset_index(drop=True, inplace=True)

    crt_end_times = stream_df['CreationStream'].loc[
    (stream_df['CreationStream']['EventName'] == 'FinishPath'), 
    ['time', 'ModelID', 'MethodID']]
    crt_end_times = pd.to_timedelta(crt_end_times.groupby(['ModelID', 'MethodID']).first()['time'], unit='s')
    crt_end_times.reset_index(drop=True, inplace=True)
    
    crt_data = {}
    for i in range(8):
        crt_data[i] = pupil_lum_df.loc[
            (pupil_lum_df['time'] > crt_start_times.loc[i]) & 
            (pupil_lum_df['time'] < crt_end_times.loc[i]), 
            ['time', 'methodID', 'modelID', 'luminance', 'pupilDiameter']]
        crt_data[i].reset_index(drop=True, inplace=True)

    creation_data = pd.concat(crt_data, names=['trial'])
    creation_data['pupil_lum_base'] = pupil_func(creation_data['luminance'], a, b, c)
    creation_data['adj_pupil'] = creation_data['pupilDiameter'] - creation_data['pupil_lum_base']
    
    return creation_data

In [62]:
def process_creation_stats(stream_df):
    stream_df['CreationStream']['ModelID'] = stream_df['CreationStream']['ModelID'].astype(float)
    stream_df['CreationStream']['MethodID'] = stream_df['CreationStream']['MethodID'].astype(float)
    stream_df['CreationStream']['EventType'] = stream_df['CreationStream']['EventType'].astype("category")

    group = stream_df['CreationStream'].groupby(['ModelID', 'MethodID'])
    creation_counts  = []

    for i in range(4):
            for j in range(0,2):
                trial = group.get_group((i,j))
                creation_counts.append(trial.groupby('EventType', observed=False).size().fillna(0))
    
    keys = [(i,j) for i in range(4) for j in range(0,2)]
    creation_stats = pd.concat(creation_counts, axis=0, keys=keys, names=['ModelID', 'MethodID']).unstack(level=2)
    creation_stats = creation_stats.drop(columns=['End', 'Start'])
    return creation_stats

In [63]:
def process_discomfort_data(stream_df):
    discomfort_values = stream_df['SurveyStream'].loc[stream_df['SurveyStream']['SurveyType'] == 'Discomfort', ['time', 'Value', 'ModelID', 'MethodID']]
    discomfort_values['time'] = pd.to_timedelta(discomfort_values['time'], unit='s')
    discomfort_values.reset_index(drop=True, inplace=True)
    return discomfort_values

In [64]:
def process_seq_data(stream_df):
    seq_values = stream_df['SurveyStream'].loc[stream_df['SurveyStream']['SurveyType'] == 'SEQ', ['time', 'Value', 'ModelID', 'MethodID']]
    seq_values['time'] = pd.to_timedelta(seq_values['time'], unit='s')
    seq_values.reset_index(drop=True, inplace=True)
    return seq_values

In [104]:
def process_ipa_calc(data):
    methods = []
    models = []
    ipa = []
    for i in range(8):
        methods.append(data.loc[i]['methodID'].iloc[i])
        models.append(data.loc[i]['modelID'].iloc[i])
        pupil = data.loc[i]['pupilDiameter']
        pupil.index = data.loc[i]['time']
        ipa.append(ipa_func(pupil))
        
    return pd.DataFrame({'methodID': methods, 'modelID': models, 'IPA': ipa})

## Load Data

Load the data from the xdf file for a single participant.

In [66]:
file = './Path_Data/ID_109.xdf'
df = import_data(file)
df['SurveyStream'] = df['SurveyStream'].replace(r'^\s*$', np.nan, regex=True).dropna()
df['CreationStream'] = df['CreationStream'].replace(r'^\s*$', np.nan, regex=True).dropna()

In [105]:
id = df['ExperimentStream']['UserID'][0]
pupil_lum_df = process_gaze_luminance_data(df)
calibration_data = process_calibration_data(pupil_lum_df, df)

x_data = calibration_data['luminance']
y_data = calibration_data['pupilDiameter']
exp_mod = Model(pupil_func)
params = exp_mod.make_params(a=1, b=4, c=0)
result = exp_mod.fit(y_data, params, x=x_data)
a = result.params['a'].value
b = result.params['b'].value
c = result.params['c'].value

pupil_data = pupil_lum_df.loc[(pupil_lum_df['methodID'] < 5), ['time', 'luminance', 'pupilDiameter']]
pupil_data.reset_index(drop=True, inplace=True)
pupil_data['time'] = pupil_data['time'] - pupil_data['time'][0]
pupil_data['pupil_lum_base'] = pupil_func(pupil_data['luminance'], a, b, c)
pupil_data['adj_pupil'] = pupil_data['pupilDiameter'] - pupil_data['pupil_lum_base']

navigation_data = process_navigation_data(pupil_lum_df, df, a, b, c)

ipa_calc_nav = process_ipa_calc(navigation_data)


In [106]:
creation_stats = process_creation_stats(df)

creation_data = process_creation_data(pupil_lum_df, df, a, b, c)

ipa_calc_crt = process_ipa_calc(creation_data)
ipa_avg_crt = ipa_calc_crt.groupby(['methodID']).mean()

creation_avg = creation_data.groupby(level=0).mean()

models_crt = creation_avg.reset_index(drop=True)
model_crt_avg = models_crt.groupby(['modelID']).mean()
model_crt_avg.drop(columns=['methodID'], inplace=True)
methods_crt = creation_avg.reset_index(drop=True)
method_crt_avg = methods_crt.groupby(['methodID']).mean()
method_crt_avg.drop(columns=['modelID'], inplace=True)

discomfort = process_discomfort_data(df)
seq = process_seq_data(df)

In [102]:
nav_trials = navigation_data.groupby(['modelID', 'methodID'])

nav_data = {}
for i in range(4):
    for j in range(2,4):
        trial = nav_trials.get_group((i, j))
        nav_data[('108',i,j)] = trial.mean()


nav_data = pd.concat(nav_data, axis=1, names=['id', 'modelID', 'methodID']).T
index = pd.MultiIndex.from_product([['108'],['A', 'B', 'C', 'D'], ['4DoF', '6DoF']], names=['id', 'modelID', 'methodID'])
nav_data.index = index
nav_data.drop(columns=['time', 'modelID', 'methodID'], inplace=True)

In [109]:
ipa_trials = ipa_calc_nav.groupby(['modelID', 'methodID'])

ipa_data = {}
for i in range(4):
    for j in range(2,4):
        trial = ipa_trials.get_group((i, j))
        ipa_data[('108',i,j)] = trial.mean()


ipa_data = pd.concat(ipa_data, axis=1, names=['id', 'modelID', 'methodID']).T
ipa_data.index = index
ipa_data.drop(columns=['modelID', 'methodID'], inplace=True)

print(ipa_data)

                           IPA
id  modelID methodID          
108 A       4DoF      0.168431
            6DoF      0.119603
    B       4DoF      0.143077
            6DoF      0.147779
    C       4DoF      0.185530
            6DoF      0.111934
    D       4DoF      0.131903
            6DoF      0.106258


In [68]:
gaze = df['GazeStream'].loc[
    (df['GazeStream']['LeftEyeIsBlinking'] == 0) & 
    (df['GazeStream']['RightEyeIsBlinking'] == 0) & 
    (df['GazeStream']['ConvergenceDistanceIsValid'] == 1) & 
    (df['GazeStream']['GazeRayIsValid'] == 1), 
    ['time', 'GazeOriginX', 'GazeOriginY', 'GazeOriginZ', 'GazeDirectionNormalizedX', 'GazeDirectionNormalizedY', 'GazeDirectionNormalizedZ']]
gaze['time'] = pd.to_timedelta(gaze['time'], unit='s')

headTransform = df['PoseStream'].loc[:, ['time', 'Head_PosX', 'Head_PosY', 'Head_PosZ', 'Head_RotX', 'Head_RotY', 'Head_RotZ', 'Head_RotW']]
headTransform['time'] = pd.to_timedelta(headTransform['time'], unit='s')

head_pupil_intersection = np.intersect1d(gaze['time'], headTransform['time'])

gaze = gaze[gaze['time'].isin(head_pupil_intersection)].reset_index(drop=True)
headTransform = headTransform[headTransform['time'].isin(head_pupil_intersection)].reset_index(drop=True)

pos_np = headTransform.loc[:, ['Head_PosX', 'Head_PosY', 'Head_PosZ']].to_numpy()
rot_np = headTransform.loc[:, ['Head_RotX', 'Head_RotY', 'Head_RotZ', 'Head_RotW']].to_numpy()
gaze_origin_np = gaze.loc[:, ['GazeOriginX', 'GazeOriginY', 'GazeOriginZ']].to_numpy()
gaze_dir_np = gaze.loc[:, ['GazeDirectionNormalizedX', 'GazeDirectionNormalizedY', 'GazeDirectionNormalizedZ']].to_numpy()

# Construct world to local transformation matrix
local_to_world = np.zeros((4, 4, len(headTransform)))
world_to_local = np.zeros((4, 4, len(headTransform)))
for i in range(len(pos_np)):
    local_to_world[:, :, i] = np.eye(4)
    q = rot_np[i]
    r = R.from_quat(q)
    t = pos_np[i]
    local_to_world[:3, :3, i] = r.as_matrix()
    local_to_world[:3, 3, i] = t
    world_to_local[:, :, i] = np.linalg.inv(local_to_world[:, :, i])

In [69]:
gaze_origin_local = np.zeros((4, len(gaze_origin_np)))

# Transform gaze origin to local coordinates
for i in range(len(gaze_origin_np)):
    gaze_origin_local[:, i] = np.append(gaze_origin_np[i], 1)
    gaze_origin_local[:, i] = np.dot(world_to_local[:, :, i], gaze_origin_local[:, i])

df_gaze = pd.DataFrame(gaze_origin_local[:3, :].T, columns=['GazeOriginX', 'GazeOriginY', 'GazeOriginZ'])

print(df_gaze.describe())
#find elements in gaze_origin_local that are greater than 1 or less than -1--these are invalied values
outliers = np.where((gaze_origin_local > 1) | (gaze_origin_local < -1))


         GazeOriginX    GazeOriginY    GazeOriginZ
count  102972.000000  102972.000000  102972.000000
mean        0.008533      -0.011716      -0.137468
std         0.431890       0.319418       0.359660
min       -17.318090     -17.868434     -14.535434
25%        -0.011203      -0.005870      -0.338260
50%         0.001796       0.002048      -0.038273
75%         0.023288       0.004919      -0.032684
max        47.367893      16.495178      29.980037


In [70]:
pupil = df['GazeStream'].loc[(df['GazeStream']['LeftEyeIsBlinking'] == 0) & (df['GazeStream']['RightEyeIsBlinking'] == 0) & (df['GazeStream']['LeftPupilDiameter'] > 0) & (df['GazeStream']['RightPupilDiameter'] > 0), ['time', 'MethodID', 'ModelID', 'LeftPupilDiameter', 'RightPupilDiameter']]
pupil['time'] = pd.to_timedelta(pupil['time'], unit='s')

lum = df['LuminanceStream'].loc[:, ['time', 'MethodID', 'ModelID', 'Luminance']]
lum['time'] = pd.to_timedelta(lum['time'], unit='s')

# Intersection of time stamps
pupil_lum_time_intersection = np.intersect1d(pupil['time'], lum['time'])

# Filter pupil and luminance data by intersection
pupil = pupil[pupil['time'].isin(pupil_lum_time_intersection)].reset_index(drop=True)
lum = lum[lum['time'].isin(pupil_lum_time_intersection)].reset_index(drop=True)

# Combined DataFrame for pupil and luminance
pupil_lum_df = pd.DataFrame({
    'time': pd.to_timedelta(pupil_lum_time_intersection, unit='s'),
    'luminance': lum['Luminance'],
    'pupilDiameter': 0.5 * (pupil['LeftPupilDiameter'] + pupil['RightPupilDiameter']),
    'methodID': pupil['MethodID'],
    'modelID': pupil['ModelID']
})


